In [28]:
import pandas as pd
from pathlib import Path
import data_utils


rename_map = {
    "نام درس": "course_title",
    "شماره درس": "course_code",
    "گروه": "section",
    "واحد": "unit",
    "نام استاد1": "professor1",
    "نام استاد2": "professor2",
    "نام استاد3": "professor3",
    "ظرفیت": "capacity",
    "تعداد ثبت نامی": "enrolled",
    "محل تشکیل": "location",
    "برنامه هفتگی": "weekly_schedule",
    "تاریخ امتحان": "exam_date",
    "ملاحظات": "notes",
    "اخطار": "warning"
}

notebook_dir = Path.cwd()
data_raw = notebook_dir.parent / "data" / "raw"
files = list(data_raw.glob("*.xls"))

all_dfs = []
for f in files:
    base = f.stem
    year = int(base[:4])
    semester = int(base[4:])
    
    df = pd.read_excel(f, header=None)
    
    # Use 2nd row as header
    df.columns = df.iloc[1]
    df = df[2:].reset_index(drop=True)
    
    df.columns = df.columns.str.strip()
    df = df.rename(columns=rename_map)
    
    df = df.dropna(how="all")
    
    df = df[~(df["course_code"].isna() & df["course_title"].isna())]
    
    for col in ["course_title", "professor1", "professor2", "professor3", "location", "notes"]:
        if col in df.columns:
            df[col] = df[col].apply(normalize_persian_text)
    
    df["year"] = year
    df["semester_code"] = semester
    
    all_dfs.append(df)

raw_df = pd.concat(all_dfs, ignore_index=True)

print(raw_df.shape)
raw_df.head(10)

(110332, 16)


1,course_title,course_code,section,unit,professor1,professor2,professor3,capacity,enrolled,location,weekly_schedule,exam_date,notes,warning,year,semester_code
0,استاتیک,20011,1,3,عبدالرضا جغتائی,None,None,NaN,37,None,NaN,-,None,NaN,1375,1
1,دینامیک,20012,1,3,میردامادی,بهشتی,None,NaN,36,None,NaN,-,None,NaN,1375,1
2,مکانیک جامدات ۱,20111,1,3,مسعود مفید,محمدمسعود علیزاده خرازی,آرمان حمیدی,NaN,40,None,NaN,-,None,NaN,1375,1
3,مکانیک جامدات ۲,20112,1,3,مسعود مفید,سیدوحید خونساری,None,NaN,20,None,NaN,-,None,NaN,1375,1
4,تئوری سازه ۱,20121,1,3,عبدالرضا جغتائی,None,None,NaN,30,None,NaN,-,None,NaN,1375,1
5,تئوری سازه ۲,20122,1,3,توکلی,مسعود مفید,None,NaN,40,None,NaN,-,None,NaN,1375,1
6,مهندسی سیستم,20191,1,2,منوچهر وزیری,None,None,NaN,24,None,NaN,-,None,NaN,1375,1
7,اقتصاد مهندسی,20197,1,3,حسین پورزاهدی,None,None,NaN,14,None,NaN,-,None,NaN,1375,1
8,مصالح‌ساختمانی‌ و تکنولوژی‌ بتن‌ و آز,20208,1,3,حاجی غفاری,خسروشاهی,None,NaN,19,None,NaN,-,None,NaN,1375,1
9,بارگذاری,20209,1,1,None,None,None,NaN,23,None,NaN,-,None,NaN,1375,1


In [26]:
# Department mapping
dept_map = {
    "20": "مهندسی عمران",
    "21": "مهندسی صنایع",
    "22": "علوم ریاضی",
    "23": "شیمی",
    "24": "فیزیک",
    "25": "مهندسی برق",
    "26": "مهندسی شیمی و نفت",
    "27": "مهندسی و علم مواد",
    "28": "مهندسی مکانیک",
    "29": "پژوهشکده سیاست‌گذاری علم، فناوری و صنعت",
    "30": "مرکز تربیت بدنی",
    "31": "مرکز زبان‌ها و زبان‌شناسی",
    "33": "مرکز آموزش مهارت‌های مهندسی",
    "34": "پژوهشکده علوم و فن‌آوری انرژی، آب و محیط زیست",
    "35": "مرکز گرافیک (مرکز آموزش مهارت‌های مهندسی)",
    "37": "مرکز معارف اسلامی و علوم انسانی",
    "38": "بیوشیمی",
    "39": "پژوهشکده الکترونیک",
    "40": "مهندسی کامپیوتر",
    "41": "گروه برنامه‌ریزی سیستم‌ها",
    "42": "گروه فلسفه علم",
    "43": "مهندسی سیستم‌های انرژی",
    "44": "مدیریت و اقتصاد",
    "45": "مهندسی هوافضا",
    "46": "مهندسی انرژی",
    "47": "پژوهشکده فناوری اطلاعات و ارتباطات پیشرفته",
    "48": "پژوهشکده علوم و فن‌آوری نانو",
    "49": "طرح مهمان تکدرس",
    "50": "دروس پایه و عمومی (پردیس کیش)",
    "51": "مهندسی صنایع (پردیس کیش)",
    "52": "مهندسی کامپیوتر (پردیس کیش)",
    "53": "مهندسی عمران (پردیس کیش)",
    "54": "مدیریت (پردیس کیش)",
    "55": "مهندسی برق (پردیس کیش)",
    "56": "مهندسی نانوفناوری (پردیس کیش)",
    "57": "مهندسی مواد (پردیس کیش)",
    "58": "مهندسی مکانیک (پردیس کیش)",
    "59": "زبان‌ها و زبان‌شناسی (پردیس کیش)",
    "61": "طرح مهمان تک درس (پردیس کیش)",
    "65": "مهندسی هوافضا (پردیس کیش)",
    "66": "مهندسی شیمی و نفت (پردیس کیش)",
    "70": "دروس پایه و عمومی (پردیس تهران)",
    "71": "طرح مهمان تک درس (پردیس تهران)",
    "73": "مهندسی عمران (پردیس تهران)",
    "76": "مهندسی نفت (پردیس تهران)",
    "77": "مهندسی مواد (پردیس تهران)",
    "78": "مهندسی مکانیک (پردیس تهران)",
    "79": "مهندسی مکاترونیک (پردیس تهران)",
    "80": "مهندسی فناوری اطلاعات (پردیس تهران)",
    "81": "مهندسی کامپیوتر (پردیس تهران)"
}

raw_df["course_code"] = raw_df["course_code"].astype(str).str.strip()

raw_df["dept_code"] = raw_df["course_code"].str[:2]

raw_df["department"] = raw_df["dept_code"].map(dept_map)

raw_df = raw_df.dropna(subset=["department"])

print(raw_df.shape)
raw_df.head(10)

(110329, 18)


1,course_title,course_code,section,unit,professor1,professor2,professor3,capacity,enrolled,location,weekly_schedule,exam_date,notes,warning,year,semester_code,dept_code,department
0,استاتیک,20011,1,3,عبدالرضا جغتائی,None,None,NaN,37,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
1,دینامیک,20012,1,3,میردامادی,بهشتی,None,NaN,36,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
2,مکانیک جامدات ۱,20111,1,3,مسعود مفید,محمدمسعود علیزاده خرازی,آرمان حمیدی,NaN,40,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
3,مکانیک جامدات ۲,20112,1,3,مسعود مفید,سیدوحید خونساری,None,NaN,20,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
4,تئوری سازه ۱,20121,1,3,عبدالرضا جغتائی,None,None,NaN,30,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
5,تئوری سازه ۲,20122,1,3,توکلی,مسعود مفید,None,NaN,40,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
6,مهندسی سیستم,20191,1,2,منوچهر وزیری,None,None,NaN,24,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
7,اقتصاد مهندسی,20197,1,3,حسین پورزاهدی,None,None,NaN,14,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
8,مصالح‌ساختمانی‌ و تکنولوژی‌ بتن‌ و آز,20208,1,3,حاجی غفاری,خسروشاهی,None,NaN,19,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران
9,بارگذاری,20209,1,1,None,None,None,NaN,23,None,NaN,-,None,NaN,1375,1,20,مهندسی عمران


In [27]:
data_processed = notebook_dir.parent / "data" / "processed"
raw_df.to_csv(data_processed / "courses_clean.csv", index=False, encoding="utf-8-sig")